In [1]:
%matplotlib inline
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from itertools import chain
from collections import Counter

## Read in and preprocess data 

In [2]:
#read all data
xls = pd.ExcelFile('./Compiled_survey.xlsx')
Jan18 = pd.read_excel(xls, 'Jan 2018')
Jan18['month'] = 'Jan 2018'
Dec17 = pd.read_excel(xls, 'Dec 2017')
Dec17['month'] = 'Dec 2017'
Nov17 = pd.read_excel(xls, 'Nov 2017')
Nov17['month'] = 'Nov 2017'
Oct17 = pd.read_excel(xls, 'Oct 2017')
Oct17['month'] = 'Oct 2017'
Sept17 = pd.read_excel(xls, 'Sept 2017')
Sept17['month'] = 'Sept 2017'
Aug17 = pd.read_excel(xls, 'August 2017')
Aug17['month'] = 'Aug 2017'
Jul17 = pd.read_excel(xls, 'July 2017')
Jul17['month'] = 'Jul 2017'
Jun17 = pd.read_excel(xls, 'June 2017')
Jun17['month'] = 'Jun 2017'
May17 = pd.read_excel(xls, 'May 2017')
May17['month'] = 'May 2017'
Apr17 = pd.read_excel(xls, 'April 2017')
Apr17['month'] = 'April 2017'
Mar17 = pd.read_excel(xls, 'Mar 2017')
Mar17['month'] = 'March 2017'
Feb17 = pd.read_excel(xls, 'Feb 2017')
Feb17['month'] = 'Feb 2017'
Jan17 = pd.read_excel(xls, 'Jan 2017')
Jan17['month'] = 'Jan 2017'

In [3]:
#combine data frames into one
data = pd.concat([Jan18, Dec17, Nov17, Oct17, Sept17, Aug17, Jul17,Jun17, May17, Apr17,
         Mar17, Feb17, Jan17])

In [4]:
#rename columns
data = data.rename(index=str, columns={"Date":"date", "What is the purpose for your visit today?": "purpose",
                                 'Were you able to complete your task today?':"task_completion",
                                 "How would you rate the level of effort you had to put forth to handle your request today?": "effort",
                                 'How did this effort compare to your expectations?':'expectations',
                                 'How likely is it that you would recommend FasTrak to a friend or colleague?':'recommend',
                                 "Additional Feedback (Optional)": "feedback",
                                "month":"month"})

In [5]:
#drop unneeded column 
data = data.drop('#', 1)

In [6]:
data["month"].value_counts()

Jul 2017      15012
Aug 2017      14988
Jan 2017      14375
Jun 2017      14068
April 2017    14068
Feb 2017      14068
May 2017      14068
March 2017    14068
Oct 2017      14061
Jan 2018      13809
Sept 2017     13237
Nov 2017      13237
Dec 2017      11579
Name: month, dtype: int64

In [7]:
data.shape

(180638, 8)

### Filter for only rows with feedback responses

In [8]:
#get only rows with feedback filled out
feedback = data[data.feedback.notnull()]

In [9]:
feedback["month"].value_counts()

Jul 2017      3056
Aug 2017      2919
Oct 2017      2721
Jan 2018      2668
Jun 2017      2664
Jan 2017      2627
March 2017    2588
Nov 2017      2551
Sept 2017     2551
April 2017    2379
Dec 2017      2202
Feb 2017      2128
May 2017      1561
Name: month, dtype: int64

In [10]:
feedback.shape

(32615, 8)

### Filter feedback for only responses with an issue

In [11]:
# filter out super positive feedback
neg_feedback = feedback[
        (feedback['expectations'] != 'Exceptional') &
        (feedback['purpose'] == 'Dispute Violation/Invoice')]

In [12]:
neg_feedback["month"].value_counts()

Oct 2017      134
Jan 2018      132
Aug 2017      129
Jul 2017      119
Jan 2017      119
Sept 2017     118
Nov 2017      118
Dec 2017      112
Jun 2017      111
March 2017    110
April 2017    108
Feb 2017       96
May 2017       61
Name: month, dtype: int64

In [13]:
neg_feedback.shape

(1467, 8)

In [14]:
# compile documents into a list
doc_set = neg_feedback.feedback.tolist()

#remove numbers from documents
doc_set = [x for x in doc_set if type(x) != int]
len(doc_set)

1467

## LDA

In [15]:
# remove words only appearing once in whole list
# https://stackoverflow.com/questions/21100903/improve-performance-remove-all-strings-in-a-big-list-appearing-only-once
c = Counter(word for x in doc_set for word in x.split())
doc_set = [' '.join(y for y in x.split() if c[y] > 1) for x in doc_set]

In [16]:
def topic_modeling(doc_set = doc_set):
    #create tokenizer
    tokenizer = RegexpTokenizer(r'\w+')

    # create English stop words list
    en_stop = get_stop_words('en')

    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    
    # list for tokenized documents in loop
    texts = []

    # loop through document list
    for i in doc_set:
        
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]

        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        
        # add tokens to list
        texts.append(stemmed_tokens) # texts is a list of list of tokens for each doc (feedback response)
    
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    # generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=6, id2word = dictionary, 
                                           passes=20, minimum_probability=0)
    return ldamodel, corpus

In [17]:
ldamodel, corpus = topic_modeling(doc_set)

In [18]:
for top in ldamodel.print_topics(num_topics=6, num_words=4):
    print(top)

(0, u'0.027*"fastrak" + 0.026*"charg" + 0.023*"account" + 0.015*"toll"')
(1, u'0.028*"toll" + 0.027*"lane" + 0.024*"t" + 0.020*"use"')
(2, u'0.050*"account" + 0.042*"vehicl" + 0.035*"violat" + 0.027*"notic"')
(3, u'0.029*"violat" + 0.023*"plate" + 0.020*"m" + 0.018*"licens"')
(4, u'0.034*"account" + 0.015*"violat" + 0.015*"time" + 0.014*"get"')
(5, u'0.030*"t" + 0.019*"bridg" + 0.019*"time" + 0.015*"call"')


## Interpretting results

https://stackoverflow.com/questions/20984841/topic-distribution-how-do-we-see-which-document-belong-to-which-topic-after-doi

In [19]:
# Assigns the topics to the documents in corpus
lda_corpus = ldamodel[corpus]

In [20]:
# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)
print threshold

0.16666666685


In [21]:
cluster1 = [j for i,j in zip(lda_corpus,doc_set) if i[0][1] > threshold]

In [22]:
cluster2 = [j for i,j in zip(lda_corpus,doc_set) if i[1][1] > threshold]

In [23]:
cluster3 = [j for i,j in zip(lda_corpus,doc_set) if i[2][1] > threshold]

In [24]:
cluster4 = [j for i,j in zip(lda_corpus,doc_set) if i[3][1] > threshold]

In [25]:
cluster5 = [j for i,j in zip(lda_corpus,doc_set) if i[4][1] > threshold]

In [26]:
cluster6 = [j for i,j in zip(lda_corpus,doc_set) if i[5][1] > threshold]

In [28]:
#print cluster1
#print cluster2
#print cluster3
#print cluster4
#print cluster5
#print cluster6